# Riiid! Answer Correctness Prediction with help lightgbm classifier with futures and parameters tuning. Submission version.

# Link on my guide notebook. 

# https://www.kaggle.com/beable/lgbm-with-parameters-features-selection

In [ ]:
import optuna
from sklearn.feature_selection import RFECV
from lightgbm import LGBMClassifier
import pandas as pd
import joblib
import numpy as np
import riiideducation
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import optuna.integration.lightgbm as lgb
from optuna.samplers import TPESampler
import matplotlib.pyplot as plt

In [ ]:
questions_df = pd.read_csv(
    '../input/riiid-test-answer-prediction/questions.csv',
    usecols=[0, 3],
    dtype={'question_id': 'int16', 'part': 'int8'}
)

model = joblib.load('../input/lgb-model-2/Final_lgb_2.joblib')
selector = joblib.load('../input/selector-2/Selector_2.joblib')

user_answers_df = pd.read_csv('../input/preprocessingcontentuser/user.csv')
content_answers_df = pd.read_csv('../input/preprocessingcontentuser/content.csv')

In [ ]:
features = [
    'timestamp',
    'user_id',
    'content_id',
    'prior_question_elapsed_time',
    'prior_question_had_explanation',

    'part',

    'mean_user_accuracy',
    'questions_skew',
    'questions_std',
    'questions_var',
    'questions_sem',

    'content_mean',
    'content_skew',
    'content_std',
    'content_var',
    'content_sem'
]

features = [features[i] for i in range(len(selector.support_)) if selector.support_[i] == True]

In [ ]:
env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    
    test_df = pd.merge(test_df, questions_df, left_on='content_id', right_on='question_id', how='left')
    test_df.drop(columns=['question_id'], inplace=True)
    test_df['prior_question_had_explanation'].fillna(bool(True), inplace=True)
    test_df = test_df.replace([-np.inf, np.inf], np.nan)
    test_df = test_df.fillna(test_df.mean())
    
    test_df = test_df[test_df['content_type_id'] != 1]
    
    test_df = test_df.merge(user_answers_df, how='left', on='user_id')
    test_df = test_df.merge(content_answers_df, how='left', on='content_id')
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].astype(bool)
    test_df = test_df.replace([-np.inf, np.inf], np.nan)
    test_df = test_df.fillna(test_df.mean())

    
    test_df['answered_correctly'] = model.predict_proba(test_df[features])[:,1]
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])
